In [1]:
# https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms


## 0) Importing Modules and MNIST Data:

In [2]:
# NumPy and SciPy modules:
import numpy as np
from scipy.ndimage import interpolation
# SciKit Learn Modules:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.linear_model  import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import PredefinedSplit
# Dask Modules (for Grid Search):
from dask.diagnostics import ProgressBar
from dask_searchcv import GridSearchCV
# Additional modules:
import random,time,gc

In [3]:
# Importing MNIST Data:
DATA_folder  = '../../Data/'
data = np.load(DATA_folder+'train_imgs.npy')
lbls = np.load(DATA_folder+'train_lbls.npy')
test_data = np.load(DATA_folder+'test_imgs.npy')
test_lbls = np.load(DATA_folder+'test_lbls.npy')

## 1) Definition Scikit-Learn Transformer Objects:

### Definition of Deskewing Transformer

In [4]:
# SKLEARN Compatible Transformer - Image Deskewing Transformation
class myDeskewer(BaseEstimator, TransformerMixin):
    
    # Transformer initialization
    def __init__(self, debug=False):
        self.debug = debug
    
    # No fitting required:
    def fit(self, X=None, y=None):        
        return self
    
    # Definition of Moments Calculation Function
    @staticmethod
    def moments(image):
        # Create mesh grids for centroid calculation:
        c0,c1 = np.mgrid[:image.shape[0],:image.shape[1]] # A trick in numPy to create a mesh grid
        # Calculate Total Image weight:
        totalImage = np.sum(image) #sum of pixels
        # Calculate image centroids:
        m0 = np.sum(c0*image)/totalImage # mu_x
        m1 = np.sum(c1*image)/totalImage # mu_y
        # Calculate image variance and covariance
        var = np.sum((c0-m0)**2*image)/totalImage      # Var(x)
        cov = np.sum((c0-m0)*(c1-m1)*image)/totalImage # Cov(x,y)
        mu_vector = np.array([m0,m1])
        alpha = cov/var
        return mu_vector, alpha

    # Definition of Deskewing Function:
    @staticmethod
    def deskew(img):
        image = img.reshape(28,28)
        c,alpha = myDeskewer.moments(image)
        # Define transformation matrix:
        affine = np.array([[1,0],[alpha,1]])
        # Define transformation offset:
        ocenter = np.array(image.shape)/2.0
        offset = c-np.dot(affine,ocenter)
        # Return affine_transformation
        return interpolation.affine_transform(image,affine,offset=offset).reshape(28*28)
    
    # Clculate Kernel Components based on obtained model
    def transform(self, X, y=None):
        
        if self.debug:
            print('Applying Deskewing Transformation!!!')
            t = time.time()
        # Preallocate memory:
        new_data = np.empty(X.shape)
        
        # Loop over each image:
        for k in range(X.shape[0]):
            new_data[k]=myDeskewer.deskew(X[k])
            
        if self.debug:
            print('Deskewing Transformation finished in: ',(time.time()-t))
        
        return new_data
    
    def fit_transform(self, X, y):
        self.fit(X,y)
        return self.transform(X)

### Definition of Batch Kernel PCA Transformer

In [5]:
# Due to high memory requirements - we cannot process the whole dataset with kPCA - we need to sample a random batch!!!

# SKLEARN Compatible Transformer - supports fit method (finding kPCA) and transform method (applying kPCA)
class myKernelPCA(BaseEstimator, TransformerMixin):
    
    # Transformer initialization (default to 250 dimensions)
    def __init__(self, n_kPCA=250, kernel = 'poly', degree=4, whiten=True, debug=False):
        self.n_kPCA    = n_kPCA # number of kernel PCA dimensions to be retained
        self.kernel    = kernel # kernel type (rbf, poly, cosine)
        self.degree    = degree # poly kernel degree
        #self.whiten    = whiten # whiten data at output layer (zero mean, un var)
        self.whiten    = whiten and (kernel!='laplacian') and (kernel!='rbf')
        self.debug     = debug  # debug flag
        self.max_smpls = 250    # max number of samples per digit
    
    # Clusters each digit and finds cluster centers:
    def fit(self, X, y):
        
        self.kPCA = KernelPCA(n_components=self.n_kPCA,kernel=self.kernel,degree=self.degree,copy_X=False,remove_zero_eig=True,random_state=1389)
        
        if self.debug:
            print('Fitting Kernel PCA !!!')
            t = time.time()
        
        # If we have less than max number of smpls - fit regular kPCA
        if X.shape[0]<(self.max_smpls*10):
            self.kPCA.fit(X)
            # If required - whiten the data (for clustering)
            if self.whiten:
                self.scaler = StandardScaler().fit(self.kPCA.transform(X))
        
        # Else randomly pick max number of smpls for each digit and fit
        else:
            # Fix the random seed (for repeatable results)
            prng = np.random.RandomState()
            prng.seed(654)
            # Check how many smps are available per digit
            l = []
            for dig in range(10):
                l.append(min(self.max_smpls,sum(y==dig)))
            # Preallocate memory
            new_data = np.empty((sum(l),X.shape[1]))
            # Fill with randomly drawn smpls
            st = 0
            for dig in range(10):
                en = st+l[dig]
                data  = X[y==dig]
                smpls = prng.choice(data.shape[0],l[dig],replace=False)
                new_data[st:en,] = data[smpls,]
                st = en
            # Fit KernelPCA to new_data
            self.kPCA.fit(new_data)
            # If required - whiten the data (for clustering)
            if self.whiten:
                self.scaler = StandardScaler().fit(self.kPCA.transform(new_data))
            # Release old vars, and collect garbage
            del data, smpls, new_data
            gc.collect()
            
        if self.debug:
            print('Kernel PCA fitted in: ',(time.time()-t))
        
        return self
    
    # Clculate Kernel Components based on obtained model
    def transform(self, X, y=None):
        
        if self.debug:
            print('Applying Kernel PCA transformation!!! (data', X.shape,')')
            t = time.time()
        # Preallocate memory:
        new_data = np.empty((X.shape[0],self.kPCA.lambdas_.shape[0]))
        st = 0
        ch = 200
        ns = X.shape[0]
        # Transform data in chunks of 200 smpls (memory hungry transformation)
        while st<ns:
            en = min(st+ch,ns)
            new_data[st:en,:] = self.kPCA.transform(X[st:en,:])
            st = en
        
        if self.whiten:
            new_data = self.scaler.transform(new_data)
                
        if self.debug:
            print('Kernel PCA transformed in: ',time.time()-t)
        
        return new_data
    
    def fit_transform(self, X, y):
        self.fit(X,y)
        return self.transform(X)

### Definition of RBF Transformer (sklearn compatible object):

In [6]:
# SKLEARN Compatible Transformer - supports fit method (custering data) and transform method (calculating fi values)
class myRBFtransformer(BaseEstimator, TransformerMixin):
    
    # Transformer initialization (default to 50 kMeans clusters)
    def __init__(self, n_centers=250, rn_ratio=0.1, cl_ratio=0.5, debug=False):
        self.n_centers = n_centers              # number of data centers to be formed per digit
        self.rn_ratio  = min(1,max(0,rn_ratio)) # random  ratio (between Random Selection and Clustering)
        self.cl_ratio  = min(1,max(0,cl_ratio)) # cluster ratio (between kMeans and Agglomerative)
        self.debug     = debug                  # debug flag
        if self.debug:
            print(self.n_centers,self.cl_ratio)
    
    # Function for data clustering, and computation of cluster center vectors and inv. sq. deviation vectors
    @staticmethod
    def form_clusters(data, n_kmeans, n_agglo, n_random, dig):

        n_km = min(n_kmeans,data.shape[0])
        n_ag = min(n_agglo ,data.shape[0])
        n_rn = min(n_random,data.shape[0])

        lbls_set = []

        if n_km>0:
            kmeans = KMeans(n_clusters=n_km, random_state=0, init='k-means++', algorithm='elkan')
            lbls_kmeans = kmeans.fit_predict(data)
            lbls_set.append(lbls_kmeans)

        if n_ag>0:
            #agglo  = SpectralClustering(n_clusters=n_ag, affinity='nearest_neighbors',eigen_solver='arpack',random_state=456,assign_labels='discretize')
            agglo  = SpectralClustering(n_clusters=n_ag, affinity='sigmoid',eigen_solver='arpack',random_state=456,assign_labels='discretize')
            lbls_agglo  = agglo.fit_predict(data)
            lbls_set.append(lbls_agglo)

        centers     = []

        # Find cluster centers and covar matrix:
        for lbls in lbls_set:
            for k in range(max(lbls)+1):
                cluster = data[lbls==k,:]
                centers.append(cluster.mean(axis=0))

            del cluster

        # Add random points as centers:
        if n_rn>0:
            # seed a new random generator (to get repeatable results for hyperparam tuning)
            prng = np.random.RandomState()
            prng.seed(dig*n_rn)
            smpls = prng.choice(data.shape[0],n_rn,replace=False)
            for s in smpls:
                centers.append(data[s,:])

        return centers;
    
    # Clusters each digit and finds cluster centers:
    def fit(self, X, y):
        
        self.centers     = [] # list of cluster center vectors
        self.num_centers = [] # list of number of centers per digit
        
        # Calc. num. of clusters per digit based on assigned ratio (ratio*(num of smpls / 10))
        n_random   = round(self.rn_ratio*self.n_centers)
        n_clusters = self.n_centers - n_random
        n_kmeans   = round(self.cl_ratio*n_clusters)
        n_agglo    = n_clusters - n_kmeans
        
        if self.debug:
            print('Clustering data ',(n_kmeans,n_agglo))
            t = time.time()
        
        # Cluster the data over each digit
        for dig in range(10):
            # print('Clustering digit: ',dig)
            data = X[y==dig,:]
            centers = myRBFtransformer.form_clusters(data, n_kmeans, n_agglo, n_random, dig)
            self.centers.extend(centers)
            self.num_centers.append(len(centers))
            
        if self.debug:
            print('Clustering time: ',(time.time()-t))
        
        del data, centers
        gc.collect()
        
        return self
    
    # Function to compute whole Fi output for given dataset (for all RBF centers)
    @staticmethod
    def fi_transform(data, all_centers):
        # data        - given dataset matrix for which to compute fi values
        # center      - list of all center vectors on which to compute fi vals

        new_data = np.empty((data.shape[0],len(all_centers)))
        st = 0
        ch = 200
        ns = data.shape[0]
        # Process data in chunks of 200 smpls (optimal speed)
        while st<ns:
            en = min(st+ch,ns)
            for k in range(len(all_centers)):
                # # SAME DEVIATION for ALL DIMENSIONS (better much better than cluster separate):
                new_data[st:en,k] = (np.square(data[st:en] - np.repeat(all_centers[k][np.newaxis,:],en-st,axis=0))).mean(axis=1)
            new_data[st:en,:] = np.exp(-np.sqrt(new_data[st:en,:]))
            st = en
            gc.collect()
        return new_data
    
    # Computes fi values (with Gaussian function) based on obtained data centers
    def transform(self, X, y=None):
        
        # Compute all Fi values:
        if self.debug:
            print('Calculating all Fi outputs !!! (data', X.shape,')')
            t = time.time()
        all_fis = myRBFtransformer.fi_transform(X, self.centers)
        
        if self.debug:
            print('Fi calculation time: ',time.time()-t)
        
        return all_fis
    
    def fit_transform(self, X, y):
        self.fit(X,y)
        return self.transform(X)

## 3) Layer Linking (pipeline):

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

n_kPCA    = 40
kernel    = 'laplacian'
n_centers = 50
rn_ratio  = 0.75
cl_ratio  = 1.0
C         = 1e3
tol       = 1e-6
debug     = False

dskw   = myDeskewer(debug=debug)
scaler = StandardScaler()
pca    = myKernelPCA(kernel=kernel,n_kPCA=n_kPCA,debug=debug)
rbf    = myRBFtransformer(n_centers=n_centers,rn_ratio=rn_ratio,cl_ratio=cl_ratio, debug=debug)
scaler2 = StandardScaler()
clsf   = LogisticRegression(tol=tol,C=C,random_state=12,solver='liblinear')
# clsf = RandomForestClassifier(n_estimators=100, max_features='sqrt' , max_depth=None, min_samples_split=2, random_state=12121)
# clsf = LogisticRegression(tol=tol,C=C,random_state=12,solver='newton-cg',multi_class='multinomial',max_iter=100)


In [8]:
pipe = Pipeline(steps=[('dskw',dskw),('scal', scaler), ('pca', pca), ('rbf',rbf),('scal2', scaler2), ('clsf',clsf)])

## 4) Grid Search:

In [9]:
# Definition of Search Space:

Ks  = [250]
n_jobs =-1
RNs = [0.6] # Random Ratio
NKs = [40]
Cs  = [1e3]
tols= [1e-6]

param_grid = [
    {
        'pca__n_kPCA':        NKs,
        'rbf__n_centers':     Ks,
        'rbf__rn_ratio':      RNs,
        'clsf__tol':          tols,
        'clsf__C':            Cs,
    }
]

In [10]:
# Definition of Train-Test Split:

test_fold = [-1]*data.shape[0]+[0]*test_data.shape[0]
ps = PredefinedSplit(test_fold)

tot_data = np.vstack((data,test_data))
tot_lbls = np.hstack((lbls,test_lbls))

del data,test_data,lbls,test_lbls
gc.collect()

0

In [11]:
grid = GridSearchCV(pipe, cv=ps, n_jobs=n_jobs, param_grid=param_grid, refit=False, return_train_score=True)

In [12]:
### Different Kernels (logreg: tol = 1e-3, C=1e3; 60k trainset):
# ------------------
# FULL kMEANS:
# 
# laplacian - 94.46%
# rbf       - 93.24%
# poly_3    - 92.95%
# sigmoid   - 91.72%
# poly_6    - 90.82%
# ------------------


In [13]:
with ProgressBar():
    grid.fit(tot_data, tot_lbls)
    
print(max(grid.cv_results_['mean_test_score'])*100)

[################################        ] | 81% Completed |  3hr  1min 49.7s


KeyboardInterrupt: 

In [ ]:
grid.param_grid

In [ ]:
print('### Parameter Grid:\n')
for k in range(len(grid.cv_results_['params'])):
    print(grid.cv_results_['params'][k])
    sc  = np.round(grid.cv_results_['mean_test_score'][k]*100,2)
    ftm = np.round(grid.cv_results_['mean_fit_time'][k])
    stm = np.round(grid.cv_results_['mean_score_time'][k])
    #print('Score:',sc,'%; Fit time:',ftm,'s; Score Time:',stm,'s;\n')
    tsc = np.round(grid.cv_results_['mean_train_score'][k]*100,2)
    print('Test Score:',sc,'%; Train Score:',tsc,'%; Fit time:',ftm,'s; Score Time:',stm,'s;\n')


In [ ]:
### Parameter Grid (for 250 centers):

# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.0}
# Test Score: 97.79 %; Train Score: 97.93 %; Fit time: 6478.0 s; Score Time: 3956.0 s;
# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.1}
# Test Score: 97.79 %; Train Score: 97.92 %; Fit time: 6528.0 s; Score Time: 3967.0 s;
# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.2}
# Test Score: 97.81 %; Train Score: 97.91 %; Fit time: 6528.0 s; Score Time: 3962.0 s;
# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.3}
# Test Score: 97.8 %; Train Score: 97.92 %; Fit time: 6456.0 s; Score Time: 3146.0 s;
# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.4}
# Test Score: 97.79 %; Train Score: 97.9 %; Fit time: 6438.0 s; Score Time: 3144.0 s;
# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.81 %; Train Score: 97.9 %; Fit time: 6435.0 s; Score Time: 3143.0 s;
# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.6}
# Test Score: 97.83 %; Train Score: 97.89 %; Fit time: 6377.0 s; Score Time: 3144.0 s;
# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.7}
# Test Score: 97.69 %; Train Score: 97.83 %; Fit time: 5213.0 s; Score Time: 1554.0 s;
# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.8}
# Test Score: 97.82 %; Train Score: 97.83 %; Fit time: 5133.0 s; Score Time: 1580.0 s;
# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.9}
# Test Score: 97.75 %; Train Score: 97.82 %; Fit time: 4976.0 s; Score Time: 1493.0 s;
# {'clsf__C': 1000.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 1}
# Test Score: 97.79 %; Train Score: 97.88 %; Fit time: 6187.0 s; Score Time: 3754.0 s;

In [ ]:
### Parameter Grid (for 500 centers):

# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.0}
# Test Score: 97.22 %; Train Score: 97.11 %; Fit time: 5351.0 s; Score Time: 103.0 s;
# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.1}
# Test Score: 97.21 %; Train Score: 97.09 %; Fit time: 5384.0 s; Score Time: 105.0 s;
# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.2}
# Test Score: 97.2 %; Train Score: 97.05 %; Fit time: 5097.0 s; Score Time: 76.0 s;
# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.3}
# Test Score: 97.21 %; Train Score: 97.07 %; Fit time: 5011.0 s; Score Time: 101.0 s;
# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.4}
# Test Score: 97.21 %; Train Score: 97.04 %; Fit time: 5164.0 s; Score Time: 106.0 s;
# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.5}
# Test Score: 97.19 %; Train Score: 97.04 %; Fit time: 5161.0 s; Score Time: 103.0 s;
# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.6}
# Test Score: 97.16 %; Train Score: 97.03 %; Fit time: 5031.0 s; Score Time: 109.0 s;
# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.7}
# Test Score: 97.2 %; Train Score: 97.01 %; Fit time: 5325.0 s; Score Time: 113.0 s;
# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.8}
# Test Score: 97.13 %; Train Score: 97.01 %; Fit time: 5128.0 s; Score Time: 86.0 s;
# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.9}
# Test Score: 97.12 %; Train Score: 96.96 %; Fit time: 3965.0 s; Score Time: 66.0 s;
# {'clsf__C': 100.0, 'clsf__tol': 1e-06, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 1}
# Test Score: 97.09 %; Train Score: 96.97 %; Fit time: 4911.0 s; Score Time: 106.0 s;

In [ ]:


# {'clsf__C': 1000000.0, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.5}
# Test Score: 98.82 %; Train Score: 99.78 %; Fit time: 75322.0 s; Score Time: 63.0 s;

In [ ]:
### Different Cs (log reg):

### Parameter Grid:

# {'clsf__C': 1000000.0, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 98.76 %; Train Score: 99.6 %; Fit time: 22907.0 s; Score Time: 4094.0 s;

# {'clsf__C': 1000000000.0, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 98.75 %; Train Score: 99.62 %; Fit time: 23035.0 s; Score Time: 3981.0 s;

# {'clsf__C': 1000000000000.0, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 98.77 %; Train Score: 99.62 %; Fit time: 23135.0 s; Score Time: 3891.0 s;

# {'clsf__C': 1000000000000000.0, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 98.74 %; Train Score: 99.65 %; Fit time: 23758.0 s; Score Time: 3606.0 s;

# {'clsf__C': 1e+18, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 98.78 %; Train Score: 99.6 %; Fit time: 13196.0 s; Score Time: 495.0 s;

# {'clsf__C': 1e+21, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 98.78 %; Train Score: 99.58 %; Fit time: 13037.0 s; Score Time: 561.0 s;


# {'clsf__C': 1000000.0, 'pca__n_kPCA': 40, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.5}
# Test Score: 98.82 %; Train Score: 99.78 %; Fit time: 75322.0 s; Score Time: 63.0 s;


In [ ]:
### Different Number of kPCA:

# {'clsf__C': 1000.0, 'pca__n_kPCA': 20, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.57 %; Train Score: 97.66 %; Fit time: 7589.0 s; Score Time: 362.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 30, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.78 %; Train Score: 97.87 %; Fit time: 7278.0 s; Score Time: 667.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 35, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.79 %; Train Score: 97.86 %; Fit time: 6955.0 s; Score Time: 3144.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 38, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.75 %; Train Score: 97.86 %; Fit time: 6748.0 s; Score Time: 3119.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 40, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.81 %; Train Score: 97.9 %; Fit time: 6951.0 s; Score Time: 973.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 42, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.78 %; Train Score: 97.85 %; Fit time: 6746.0 s; Score Time: 3123.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 45, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.72 %; Train Score: 97.81 %; Fit time: 6742.0 s; Score Time: 3111.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 50, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.78 %; Train Score: 97.79 %; Fit time: 6582.0 s; Score Time: 1326.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 100, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.42 %; Train Score: 97.42 %; Fit time: 5462.0 s; Score Time: 3383.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 150, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.25 %; Train Score: 97.24 %; Fit time: 5212.0 s; Score Time: 3358.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 200, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.13 %; Train Score: 97.1 %; Fit time: 5211.0 s; Score Time: 3306.0 s;

# {'clsf__C': 1000.0, 'pca__n_kPCA': 250, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 96.98 %; Train Score: 96.99 %; Fit time: 5207.0 s; Score Time: 3358.0 s;

In [ ]:
### LOGISTIC REGRESSION:

# {'logreg__C': 1000.0, 'pca__n_kPCA': 29, 'rbf__n_centers': 50, 'rbf__rn_ratio': 0.5}
# Test Score: 96.88 %; Train Score: 96.7 %; Fit time: 1306.0 s; Score Time: 72.0 s;

# {'logreg__C': 1000.0, 'pca__n_kPCA': 29, 'rbf__n_centers': 100, 'rbf__rn_ratio': 0.5}
# Test Score: 97.35 %; Train Score: 97.29 %; Fit time: 2531.0 s; Score Time: 81.0 s;

# {'logreg__C': 1000.0, 'pca__n_kPCA': 29, 'rbf__n_centers': 150, 'rbf__rn_ratio': 0.5}
# Test Score: 97.61 %; Train Score: 97.58 %; Fit time: 3725.0 s; Score Time: 70.0 s;

# {'logreg__C': 1000.0, 'pca__n_kPCA': 29, 'rbf__n_centers': 200, 'rbf__rn_ratio': 0.5}
# Test Score: 97.71 %; Train Score: 97.74 %; Fit time: 4742.0 s; Score Time: 59.0 s;

# {'logreg__C': 1000.0, 'pca__n_kPCA': 29, 'rbf__n_centers': 250, 'rbf__rn_ratio': 0.5}
# Test Score: 97.75 %; Train Score: 97.85 %; Fit time: 4741.0 s; Score Time: 50.0 s;

# {'logreg__C': 1000.0, 'pca__n_kPCA': 29, 'rbf__n_centers': 400, 'rbf__rn_ratio': 0.5}
# Test Score: 97.93 %; Train Score: 98.14 %; Fit time: 8134.0 s; Score Time: 69.0 s;

# {'logreg__C': 1000.0, 'pca__n_kPCA': 29, 'rbf__n_centers': 500, 'rbf__rn_ratio': 0.5}
# Test Score: 98.04 %; Train Score: 98.25 %; Fit time: 9869.0 s; Score Time: 63.0 s;

In [ ]:
# ### Parameter Grid:

# {'logreg__C': 1000.0, 'pca__n_kPCA': 29, 'rbf__n_centers': 10, 'rbf__rn_ratio': 0.0}
# Test Score: 95.64 %; Train Score: 95.28 %; Fit time: 339.0 s; Score Time: 37.0 s;

# ### Parameter Grid:

# {'logreg__C': 1000.0, 'pca__n_kPCA': 29, 'rbf__n_centers': 10, 'rbf__rn_ratio': 0.0}
# Test Score: 95.64 %; Train Score: 95.28 %; Fit time: 336.0 s; Score Time: 37.0 s;